In [1]:
import polars as pl

In [2]:
train_df = pl.read_csv('train.csv')
train_df

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str,bool
"""0001_01""","""Europa""",false,"""B/0/P""","""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,"""Maham Ofracculy""",false
"""0002_01""","""Earth""",false,"""F/0/S""","""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,"""Juanna Vines""",true
"""0003_01""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",58.0,true,43.0,3576.0,0.0,6715.0,49.0,"""Altark Susent""",false
"""0003_02""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",33.0,false,0.0,1283.0,371.0,3329.0,193.0,"""Solam Susent""",false
"""0004_01""","""Earth""",false,"""F/1/S""","""TRAPPIST-1e""",16.0,false,303.0,70.0,151.0,565.0,2.0,"""Willy Santantines""",true
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9276_01""","""Europa""",false,"""A/98/P""","""55 Cancri e""",41.0,true,0.0,6819.0,0.0,1643.0,74.0,"""Gravior Noxnuther""",false
"""9278_01""","""Earth""",true,"""G/1499/S""","""PSO J318.5-22""",18.0,false,0.0,0.0,0.0,0.0,0.0,"""Kurta Mondalley""",false
"""9279_01""","""Earth""",false,"""G/1500/S""","""TRAPPIST-1e""",26.0,false,0.0,0.0,1872.0,1.0,0.0,"""Fayey Connon""",true


In [3]:
train_df = pl.sql("Select * replace(case when Transported = 'false' then 0 when Transported = 'true' then 1 else 2 end as Transported) from train_df").collect()
train_df = pl.sql("""
    SELECT 
        * exclude (Name, Cabin), 
        SPLIT_PART(Cabin, '/', 1) as Cabin_First, 
        CAST(SPLIT_PART(Cabin, '/', 2) AS INTEGER) as Cabin_Second, 
        SPLIT_PART(Cabin, '/', 3) as Cabin_Third,
        SPLIT_PART(PassengerId, '_', 1) as Family_Id,
        (RoomService + FoodCourt + ShoppingMall + Spa + VRDeck) as AllSpending 
    FROM train_df
""").collect()

family_counts_summary = train_df.group_by("Family_Id").agg(
    pl.count().alias("Family_Count")
)

train_df = pl.sql("select * from train_df left join family_counts_summary using (Family_Id)").collect()
train_df = pl.sql('select * exclude("Family_Id:family_counts_summary") from train_df').collect()

#train_df = train_df.drop('Family_Id')
train_df

C:\Users\Gahtan Syarif Nahdi\AppData\Local\Temp\ipykernel_17388\3598407946.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("Family_Count")


PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_First,Cabin_Second,Cabin_Third,Family_Id,AllSpending,Family_Count
str,str,bool,str,f64,bool,f64,f64,f64,f64,f64,i32,str,i32,str,str,f64,u64
"""0001_01""","""Europa""",false,"""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,0,"""B""",0,"""P""","""0001""",0.0,1
"""0002_01""","""Earth""",false,"""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,1,"""F""",0,"""S""","""0002""",736.0,1
"""0003_01""","""Europa""",false,"""TRAPPIST-1e""",58.0,true,43.0,3576.0,0.0,6715.0,49.0,0,"""A""",0,"""S""","""0003""",10383.0,2
"""0003_02""","""Europa""",false,"""TRAPPIST-1e""",33.0,false,0.0,1283.0,371.0,3329.0,193.0,0,"""A""",0,"""S""","""0003""",5176.0,2
"""0004_01""","""Earth""",false,"""TRAPPIST-1e""",16.0,false,303.0,70.0,151.0,565.0,2.0,1,"""F""",1,"""S""","""0004""",1091.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9276_01""","""Europa""",false,"""55 Cancri e""",41.0,true,0.0,6819.0,0.0,1643.0,74.0,0,"""A""",98,"""P""","""9276""",8536.0,1
"""9278_01""","""Earth""",true,"""PSO J318.5-22""",18.0,false,0.0,0.0,0.0,0.0,0.0,0,"""G""",1499,"""S""","""9278""",0.0,1
"""9279_01""","""Earth""",false,"""TRAPPIST-1e""",26.0,false,0.0,0.0,1872.0,1.0,0.0,1,"""G""",1500,"""S""","""9279""",1873.0,1


In [4]:
train_df.describe()

statistic,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_First,Cabin_Second,Cabin_Third,Family_Id,AllSpending,Family_Count
str,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,f64,f64
"""count""","""8693""","""8492""",8476.0,"""8511""",8514.0,8490.0,8512.0,8510.0,8485.0,8510.0,8505.0,8693.0,"""8494""",8494.0,"""8494""","""8693""",7785.0,8693.0
"""null_count""","""0""","""201""",217.0,"""182""",179.0,203.0,181.0,183.0,208.0,183.0,188.0,0.0,"""199""",199.0,"""199""","""0""",908.0,0.0
"""mean""",null,null,0.358306,null,28.82793,0.023439,224.687617,458.077203,173.729169,311.138778,304.854791,0.503624,null,600.367671,null,null,1484.601541,2.035546
"""std""",null,null,null,null,14.489021,null,666.717663,1611.48924,604.696458,1136.705535,1145.717189,0.500016,null,511.867226,null,null,2845.288241,1.596347
"""min""","""0001_01""","""Earth""",0.0,"""55 Cancri e""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""A""",0.0,"""P""","""0001""",0.0,1.0
"""25%""",null,null,null,null,19.0,null,0.0,0.0,0.0,0.0,0.0,0.0,null,167.0,null,null,0.0,1.0
"""50%""",null,null,null,null,27.0,null,0.0,0.0,0.0,0.0,0.0,1.0,null,427.0,null,null,736.0,1.0
"""75%""",null,null,null,null,38.0,null,47.0,76.0,27.0,59.0,46.0,1.0,null,999.0,null,null,1486.0,3.0
"""max""","""9280_02""","""Mars""",1.0,"""TRAPPIST-1e""",79.0,1.0,14327.0,29813.0,23492.0,22408.0,24133.0,1.0,"""T""",1894.0,"""S""","""9280""",35987.0,8.0


In [5]:
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'Cabin_First', 'Cabin_Third', 'VIP', 'Family_Id']


In [6]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
X = pl.sql("select * exclude (PassengerId, Transported) from train_df").collect().to_pandas()
y = train_df['Transported'].to_pandas()

for col in cat_features:
    X[col] = X[col].astype("string").fillna("Null")

In [8]:
X

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_First,Cabin_Second,Cabin_Third,Family_Id,AllSpending,Family_Count
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0.0,P,0001,0.0,1
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0.0,S,0002,736.0,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0.0,S,0003,10383.0,2
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0.0,S,0003,5176.0,2
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1.0,S,0004,1091.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,A,98.0,P,9276,8536.0,1
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,G,1499.0,S,9278,0.0,1
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,G,1500.0,S,9279,1873.0,1
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,E,608.0,S,9280,4637.0,2


In [9]:
y

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 8693, dtype: int32

In [10]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1986)

# Create and train the CatBoostClassifier
model = CatBoostClassifier(cat_features=cat_features, depth=7)  # Set verbose to 0 to suppress output
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Learning rate set to 0.023581
0:	learn: 0.6816914	total: 199ms	remaining: 3m 18s
1:	learn: 0.6743995	total: 218ms	remaining: 1m 48s
2:	learn: 0.6632161	total: 261ms	remaining: 1m 26s
3:	learn: 0.6533930	total: 303ms	remaining: 1m 15s
4:	learn: 0.6422349	total: 347ms	remaining: 1m 8s
5:	learn: 0.6303964	total: 388ms	remaining: 1m 4s
6:	learn: 0.6208951	total: 432ms	remaining: 1m 1s
7:	learn: 0.6112202	total: 473ms	remaining: 58.7s
8:	learn: 0.6035073	total: 516ms	remaining: 56.8s
9:	learn: 0.5951294	total: 559ms	remaining: 55.3s
10:	learn: 0.5876274	total: 599ms	remaining: 53.9s
11:	learn: 0.5802710	total: 642ms	remaining: 52.8s
12:	learn: 0.5736672	total: 682ms	remaining: 51.8s
13:	learn: 0.5677146	total: 726ms	remaining: 51.1s
14:	learn: 0.5612966	total: 767ms	remaining: 50.4s
15:	learn: 0.5555814	total: 811ms	remaining: 49.9s
16:	learn: 0.5500732	total: 854ms	remaining: 49.4s
17:	learn: 0.5451320	total: 895ms	remaining: 48.8s
18:	learn: 0.5418581	total: 926ms	remaining: 47.8s
19:	lea

In [11]:
model = CatBoostClassifier(cat_features=cat_features, depth=7)  # Set verbose to 0 to suppress output
model.fit(X, y)

Learning rate set to 0.025939
0:	learn: 0.6810498	total: 43.8ms	remaining: 43.7s
1:	learn: 0.6668473	total: 89ms	remaining: 44.4s
2:	learn: 0.6545235	total: 134ms	remaining: 44.5s
3:	learn: 0.6427208	total: 184ms	remaining: 45.8s
4:	learn: 0.6330245	total: 237ms	remaining: 47.1s
5:	learn: 0.6239156	total: 284ms	remaining: 47.1s
6:	learn: 0.6136311	total: 329ms	remaining: 46.7s
7:	learn: 0.6054552	total: 376ms	remaining: 46.7s
8:	learn: 0.5974541	total: 421ms	remaining: 46.4s
9:	learn: 0.5901999	total: 471ms	remaining: 46.7s
10:	learn: 0.5817742	total: 516ms	remaining: 46.4s
11:	learn: 0.5757013	total: 561ms	remaining: 46.2s
12:	learn: 0.5686726	total: 607ms	remaining: 46.1s
13:	learn: 0.5613713	total: 655ms	remaining: 46.1s
14:	learn: 0.5564478	total: 702ms	remaining: 46.1s
15:	learn: 0.5502202	total: 747ms	remaining: 46s
16:	learn: 0.5466682	total: 792ms	remaining: 45.8s
17:	learn: 0.5412607	total: 840ms	remaining: 45.8s
18:	learn: 0.5365311	total: 888ms	remaining: 45.9s
19:	learn: 0.

In [12]:
test_df = pl.read_csv('test.csv')
test_df = pl.sql("Select * replace(case when Transported = 'false' then 0 when Transported = 'true' then 1 else 2 end as Transported) from test_df").collect()
test_df = pl.sql("""
    SELECT 
        * exclude (Name, Cabin), 
        SPLIT_PART(Cabin, '/', 1) as Cabin_First, 
        CAST(SPLIT_PART(Cabin, '/', 2) AS INTEGER) as Cabin_Second, 
        SPLIT_PART(Cabin, '/', 3) as Cabin_Third,
        SPLIT_PART(PassengerId, '_', 1) as Family_Id,
        (RoomService + FoodCourt + ShoppingMall + Spa + VRDeck) as AllSpending 
    FROM test_df
""").collect()

family_counts_summary = test_df.group_by("Family_Id").agg(
    pl.len().alias("Family_Count")
)

test_df = pl.sql("select * from test_df left join family_counts_summary using (Family_Id)").collect()
test_df = pl.sql('select * exclude("Family_Id:family_counts_summary") from test_df').collect()

#test_df = test_df.drop('Family_Id')
test_df

PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_First,Cabin_Second,Cabin_Third,Family_Id,AllSpending,Family_Count
str,str,bool,str,f64,bool,f64,f64,f64,f64,f64,str,i32,str,str,f64,u64
"""0013_01""","""Earth""",true,"""TRAPPIST-1e""",27.0,false,0.0,0.0,0.0,0.0,0.0,"""G""",3,"""S""","""0013""",0.0,1
"""0018_01""","""Earth""",false,"""TRAPPIST-1e""",19.0,false,0.0,9.0,0.0,2823.0,0.0,"""F""",4,"""S""","""0018""",2832.0,1
"""0019_01""","""Europa""",true,"""55 Cancri e""",31.0,false,0.0,0.0,0.0,0.0,0.0,"""C""",0,"""S""","""0019""",0.0,1
"""0021_01""","""Europa""",false,"""TRAPPIST-1e""",38.0,false,0.0,6652.0,0.0,181.0,585.0,"""C""",1,"""S""","""0021""",7418.0,1
"""0023_01""","""Earth""",false,"""TRAPPIST-1e""",20.0,false,10.0,0.0,635.0,0.0,0.0,"""F""",5,"""S""","""0023""",645.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9266_02""","""Earth""",true,"""TRAPPIST-1e""",34.0,false,0.0,0.0,0.0,0.0,0.0,"""G""",1496,"""S""","""9266""",0.0,2
"""9269_01""","""Earth""",false,"""TRAPPIST-1e""",42.0,false,0.0,847.0,17.0,10.0,144.0,null,null,null,"""9269""",1018.0,1
"""9271_01""","""Mars""",true,"""55 Cancri e""",null,false,0.0,0.0,0.0,0.0,0.0,"""D""",296,"""P""","""9271""",0.0,1


In [13]:
X = pl.sql("select * exclude (PassengerId) from test_df").collect().to_pandas()

for col in cat_features:
    X[col] = X[col].astype("string").fillna("Null")

In [14]:
y_final = model.predict(X)
y_final

array([0, 0, 1, ..., 1, 1, 1])

In [15]:
new_column_df = pl.DataFrame({'Transported': y_final})
existing_df = test_df[['PassengerId']]
final_df = existing_df.with_columns(new_column_df['Transported'])
final_df = pl.sql("Select * replace(case when Transported = 1 then 'True' else 'False' end as Transported) from final_df").collect()
final_df.write_csv("catboost_results.csv")
final_df

PassengerId,Transported
str,str
"""0013_01""","""False"""
"""0018_01""","""False"""
"""0019_01""","""True"""
"""0021_01""","""True"""
"""0023_01""","""False"""
…,…
"""9266_02""","""True"""
"""9269_01""","""False"""
"""9271_01""","""True"""


In [16]:
import pandas as pd
# After fitting your model
importances = model.feature_importances_
features = X_train.columns  # or X_train.columns

# Create a DataFrame for easy sorting and visualization
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_df 

,Feature,Importance
9,VRDeck,10.759282
8,Spa,10.732767
10,Cabin_First,10.464288
11,Cabin_Second,10.403449
1,CryoSleep,8.458708
6,FoodCourt,7.328880
5,RoomService,7.264712
0,HomePlanet,6.880557
3,Age,6.432916
12,Cabin_Third,5.019482
